In [1]:
import pandas as pd

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words("indonesian") + list(punctuation)

# Import Data

In [3]:
df = pd.read_csv("data/kompas.csv")
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


# Extract TFIDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
tfidf = TfidfVectorizer(tokenizer=word_tokenize, stop_words=sw_indo, ngram_range=(1, 2))
tfidf_matrix = tfidf.fit_transform(df.teks)

/Users/wiradharmakencanaputra/miniconda3/envs/jcopml/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


# Document Siimilarity

In [9]:
from sklearn.metrics.pairwise import cosine_distances

In [12]:
dist = cosine_distances(tfidf_matrix[0], tfidf_matrix)
dist

array([[0.        , 0.99141672, 0.98939957, ..., 0.99143713, 0.99322192,
        0.98486659]])

In [13]:
dist.argsort()

array([[   0,  144,  215, ..., 1593, 1131,  932]])

In [15]:
df.teks[0][:200]

'Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan'

In [16]:
df.teks[144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [17]:
df.teks[215][:200]

'Kuasa Hukum Ginandjar Bertahan di Rutan\nSejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pag'

In [18]:
df.teks[932][:200]

'Pusat Pengolahan Ikan Asin Lhokseumawe Terbakar\nPusat pengolahan ikan asin di kawasan Pasar Ikan Lhokseumawe (Aceh) terbakar bersama berton-ton stok ikan asin Sabtu (13/1) siang. Sedikitnya 25 gudang '

In [19]:
def search(query, tfidf, tfidf_matrix, df):
    vec = tfidf.transform([query])
    dist = cosine_distances(vec, tfidf_matrix)
    best_idx = dist.argsort()[0, :10]
    return df.teks[best_idx]

In [20]:
result = search("""
Kita masih cukup berprihatin masih ada penambahan kasus positif sebanyak 106 kasus, jadi total ada 2.092 kasus positif ," ujar juru bicara pemerintah untuk penanganan virus korona Achmad Yurianto dalam konferensi pers di Gedung Badan Nasional Penanggulangan Bencana (BNPB), Jakarta, Sabtu, 4 April 2020. Yurianto mengatakan jumlah pasien positif korona yang sembuh juga bertambah 16 orang. Total ada 150 orang sembuh dari wabah penyakit yang pertama kali ditemukan di Wuhan, Tiongkok, itu.
"Kita bersyukur mereka yang sembuh kondisinya sudah bagus, dan tak perlu dikhawatirkan menularkan lagi," kata dia.
Jumlah pasien yang meninggal akibat covid-19 bertambah 10 orang. "Sehingga totalnya ada 191 yang meninggal," ujar dia.
""", tfidf, tfidf_matrix, df)

In [22]:
result.iloc[0]

'Masyarakat Harus Waspadai Leptospirosis\n\nDinas Kesehatan DKI Jakarta bekerja sama dengan Departemen Kesehatan dan Badan Penelitian dan Pengembangan (Balitbang) Veteriner Bogor membentuk tim untuk mengkaji lebih dalam temuan kasus penyakit leptospirosis, yang diderita sejumlah pasien pascabanjir ini. Berkaitan dengan itu, masyarakat tidak perlu khawatir. Masyarakat hanya dituntut untuk waspada dengan menjalankan pola hidup sehat. \n\nDemikian penjelasan Kepala Seksi Promosi Kesehatan/ Hubungan Masyarakat Dinas Kesehatan DKI Jakarta Evi Zelfino, Rabu (6/3). "Walaupun penyakit lama, penyakit ini memang tidak familiar dengan kita. Dia meruyak karena banjir melanda kita," kata Evi.  \n\nSehari sebelumnya, diketahui bahwa delapan dari 17 warga DKI Jakarta meninggal dunia karena terserang penyakit leptospirosis. (Kompas, 6/3)       \n\nMenurut data Dinas Kesehatan DKI Jakarta hingga Rabu, sebanyak 21 warga terjangkit penyakit leptospirosis, delapan di antaranya meninggal dunia. Menanggapi 

# Keyword Extraction

In [23]:
vocab = tfidf.get_feature_names()
vocab[-10:]

['zuniga memilih',
 'zunnatul',
 'zunnatul mafruhah',
 'zurich',
 'zurich northholt',
 'zw',
 'zw suparman',
 'zw tim',
 'zx',
 'zx diserbu']

In [24]:
sorted_tfidf = tfidf_matrix[0].toarray().argsort()
sorted_tfidf

array([[274201, 365453, 365452, ..., 386361, 436633, 169209]])

In [27]:
[vocab[idx] for idx in reversed(sorted_tfidf[0, -10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

# Sanity Check

In [28]:
def extract_keywords_tfidf(doc, tfidf, topk=10):
    vec = tfidf.transform([doc])
    vocab = tfidf.get_feature_names()
    
    sorted_tfidf = vec.toarray().argsort()
    return [vocab[idx] for idx in reversed(sorted_tfidf[0, -topk:])]

In [29]:
text = """
Amerikanisasi
di luar amerika serikat , amerikanisasi adalah istilah untuk menyebut pengaruh yang dimiliki amerika serikat di negara lain , contohnya budaya masyarakat , masakan , teknologi , praktik bisnis , atau teknik politiknya .
istilah ini sudah digunakan sejak tahun 1907 .
di amerika serikat sendiri , kata amerikanisasi berarti proses akulturasi adat dan nilai-nilai amerika serikat oleh imigran atau penduduk yang dianeksasi ( misalnya california ) .
hollywood ( industri film dan televisi amerika serikat ) mendominasi sebagian besar pasar media dunia .
inilah medium yang dipakai orang-orang di seluruh dunia untuk melihat gaya , adat , suasana , dan cara hidup amerika serikat .
beberapa perusahaan komputer terbesar di dunia juga berkantor pusat di amerika serikat , misalnya microsoft , apple , dell , dan ibm .
sebagian besar perangkat lunak yang dibeli di seluruh dunia juga diciptakan oleh perusahaan-perusahaan amerika serikat .
carayannis dan campbell menulis bahwa , " secara global , amerika serikat memiliki posisi yang sangat kuat di sektor perangkat lunak .
"
di jerman pada tahun 1920 - an , gerakan efisiensi amerika serikat disebut " rasionalisasi " .
ini adalah gerakan sosial dan ekonomi yang cukup kuat .
gerakan ini memandang secara berlebihan model - model amerika serikat , khususnya fordisme .
" rasionalisasi " berarti produktivitas yang lebih tinggi dan efisiensi yang lebih besar , menjanjikan bahwa ilmu pengetahuan akan memberi kemakmuran .
secara umum , rasionalisasi menjanjikan tingkatan modernitas baru dan diterapkan pada produksi dan konsumsi ekonomi dan administrasi publik .
berbagai versi rasionalisasi diperkenalkan oleh para industrialis dan demokrat sosial , insinyur dan arsitek , pendidik dan akademisi , feminis dan pekerja sosial kelas menengah , pejabat pemerintahan dan politikus partai .
sebagai ideologi dan praktik , rasionalisasi menantang dan mengubah tidak hanya mesin , pabrik , dan perusahaan bisnis , tetapi juga kehidupan warga jerman kelas menengah dan pekerja .
selama 15 tahun sejak 1950 sampai 1965 , investasi amerika serikat di eropa melonjak 800 % menjadi $ 13,9 miliar dan di komunitas ekonomi eropa naik 10 kali lipat menjadi $ 6,25 miliar .
pangsa eropa di dunia investasi amerika serikat naik dari 15 % ke 28 % .
investasi adalah model amerikanisasi yang paling jelas dan sering diperbincangkan .
meski begitu , investasi amerika di eropa hanya mewakili 5 % dari total investasi eropa dan perusahaan milik amerika di komunitas ekonomi eropa hanya mempekerjakan 2 atau 3 % total tenaga kerja .
alasan yang mendasari investasi amerika serikat ini bukan lagi biaya produksi rendah , pertumbuhan ekonomi cepat , atau laba tinggi di eropa , tetapi keinginan untuk mempertahankan posisi bersaing yang didasari superioritas teknologi amerika serikat .
penolakan investasi amerika serikat awalnya muncul di Perancis , kemudian menyebar ke negara-negara eropa lainnya .
opini publik mulai menolak iklan dan metode bisnis amerika , kebijakan personel , dan pemakaian bahasa inggris oleh perusahaan-perusahaan amerika .
kritik juga diarahkan ke sistem mata uang internasional yang dituduh sebagai penyebab kecenderungan inflasi akibat posisi dolar a.s. yang dominan .
akan tetapi pada 1970 - an , investasi eropa di a.s. meningkat lebih cepat daripada sebelumnya .
geir lundestad menemukan bahwa persepsi " amerika serikat membeli eropa " juga mulai berkurang .
berghahn ( 2010 ) menganalisis perdebatan mengenai manfaat konsep ' amerikanisasi ' dan ' westernisasi ' .
ia meninjau penelitian terkini mengenai hubungan eropa - amerika serikat selama perang dingin yang memengaruhi dampak budaya amerika serikat di eropa .
ia kemudian mendiskusikan penelitian yang relevan terhadap subjek ini di bidang sejarah ekonomi dan bisnis .
secara keseluruhan , artikelnya mencoba memaparkan bahwa orang-orang yang menerapkan konsep ' amerikanisasi ' ke penelitian mereka mengenai sejarah budaya dan / atau ekonomi sudah sadar tentang kerumitan hubungan lintas atlantik pada masa tersebut , apakah dipandang sebagai pertukaran dua arah atau proses sirkulasi .
"""

In [30]:
extract_keywords_tfidf(text, tfidf)

['amerika',
 'amerika serikat',
 'serikat',
 'eropa',
 'investasi amerika',
 'rasionalisasi',
 'investasi',
 'ekonomi',
 'ekonomi eropa',
 'bisnis']